<a href="https://colab.research.google.com/github/alejandrogolfe/03MIAR-Algoritmos-de-optimizacion-/blob/main/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
**Nombre y Apellidos**:   ALEJANDRO GOLFE SAN MARTÍN<br>
**Url**: https://github.com/alejandrogolfe/03MIAR-Algoritmos-de-optimizacion<br>
**Url drive**: https://drive.google.com/file/d/11U3ooLroNhBe1nNuPnhmcUcn60MIQgRr/view?usp=sharing<br>
**Problema escogido**:

- Combinar cifras y operaciones

**Descripción del problema**:

Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/).

Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una
cantidad dada. Un ejemplo sería para obtener el 4:




                                        

# 1. (*) ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.



- **Sin restricciones**, es decir, pudiendo escoger cualquier número en cualquier cantidad y escogiendo la cantidad de operadores que se desee, podría llegarse a la conclusión de que el espacio de soluciones sería inexplorable al completo ya que resultaría de tamaño infinito. Esto se debe a que el conjunto de números enteros es infinito y siempre podrían evaluarse infinitas combinaciones.

- **Con restricciones** esto cambia ya que se limita el número de posibles números a elegir y su cantidad. En este caso se escogen 5 números en el rango de 1 a 9 y con cuatro operadores matemáticos (multiplicación,división,suma y resta). Con estos 9 números se pueden llevar a cabo N combinaciones de 5 números y con los 4 operadores se pueden llevar a cabo M combinaciones de los 4 elementos. El número total de posibilidades se obtendrá multiplicando estos valores MxN. A continuación, se muestra su cálculo:
<ul>
<li><font size="3"> M=</font><font size="5"> $\frac{n!}{(n-k)!}$ = $\frac{9!}{(9-5)!}$ =</font> <font size="3"> 15120</font></li> 
<li><font size="3"> N=</font><font size="3"> 4!= 24</font> </li> 
<li><font size="3"> M*N=</font><font size="3"> 362880 iteraciones</font> </li>    
 

</ul>

A continuación, se muestra un fragmento de código en el que se demuestra que ese es el número de iteraciones para el problema dado:



In [ ]:
from itertools import permutations
import random

lista_numeros=range(1,10)
lista_signos='+-/*'

maximo=0
minimo=1000000


cuenta=0

for numeros in permutations(lista_numeros,5):
    for signos in permutations(lista_signos,4):
        expresion=str(numeros[0])+signos[0]+str(numeros[1])+signos[1]+str(numeros[2])+signos[2]+str(numeros[3])+signos[3]+str(numeros[4])
        valor=eval(expresion)
        if(valor%1==0):
            if(valor>maximo):
                maximo=valor
                expresion_maxima=expresion
            if(valor<minimo):
                minimo=valor
                expresion_minima=expresion
        cuenta=cuenta+1
print('El número de iteraciones es: '+str(cuenta))
print('El valor máximo que se puede representar es: '+str(maximo))
print('Su primera expresión encontrada es: '+expresion_maxima)
print('El valor mínimo que se puede representar es: '+str(minimo))
print('Su primera expresión encontrada es: '+expresion_minima)


El número de iteraciones es: 362880
El valor máximo que se puede representar es: 77.0
Su primera expresión encontrada es: 7/1-2+8*9
El valor mínimo que se puede representar es: -69.0
Su primera expresión encontrada es: 1+4/2-8*9


Adicionalmente surge la siguiente duda, **¿Se pueden representar todos los valores enteros entre el valor máximo y mínimo?**. Para averiguarlo, se define la función que implementa el algoritmo con una ligera modificación para comprobar si una combinación para cada valor dado.

In [ ]:

def algoritmo_comprobar_representacion(numero_deseado):
    lista_numeros=[1,2,3,4,5,6,7,8,9]
    random.shuffle(lista_numeros)
    lista_signos='+-/*'
    encontrado=0
    iteraciones=0
    for numeros in permutations(lista_numeros,5):
        if(encontrado):
            break
        for signos in permutations(lista_signos,4):
            expresion=str(numeros[0])+signos[0]+str(numeros[1])+signos[1]+str(numeros[2])+signos[2]+str(numeros[3])+signos[3]+str(numeros[4])
            valor=eval(expresion)
            iteraciones+=1
            if(valor==numero_deseado):
                encontrado=1
                expresion_definitiva=expresion
                break
    if(encontrado):
        return True
    else:
        return False

for num in range(int(minimo),int(maximo+1)):
    if(algoritmo_comprobar_representacion(num)==False):
        print('Hay un número que no se puede representar')
        break
    else:
        if(num==maximo):
            print('Todos los números se pueden representar')

Todos los números se pueden representar


# 2.Modelo para el espacio de soluciones<br>
## (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


La estructura que mejor se adapta a las características del problema es la estructura de árbol. Ya que se se cuenta con un elevado número de permutaciones, dentro de cada subcombinación se pueden obtener multiples nodos hijos. Una vez identificada esta estructura se ha osbervado que muchas combinaciones no merecen ser exploradas porque no son factibles. Por ejemplo, una combinación que construye una división que no tiene como resultado un número entero, ya es descartable. En este trabajo se va a aplicar la técnica de ramificación y poda, intentando explorar en primer lugar los nodos de mayor interés y descartando aquellos que no son factibles.

# 3. Según el modelo para el espacio de soluciones<br>
## (*)¿Cual es la función objetivo?

En este caso la función objetivo es que el resultado de la expresión matématica construida sea igual al número deseado por el usuario.

## (*)¿Es un problema de maximización o minimización?

En este caso no se trata de ninguno de los dos ya que no se desea obtener una función de mayor o menor coste, simplemente se desea obtener una solución válida para el problema planteado. Esto también quiere decir que para un número dado, es posible que haya múltiples soluciones, pero la finalidad de este trabajo es garantizar que se encuentra al menos una de ellas ya que toda solución que cumple las condiciones es válida.

# 4. Diseña un algoritmo para resolver el problema por fuerza bruta

Aunque en apartados anteriores ya se ha mostrado el algoritmo de fuerza bruta, en este apartado se implementa la función que lo lleva a cabo proporcionando el número de iteraciones necesarias hasta llegar a la solución y el valor de la expresión. En este caso se propone un algoritmo general que escoge entre los números dados en la lista de entrada.

In [ ]:
def algoritmo_fuerza_bruta(lista_numeros,numero_deseado):
    lista_signos='+-/*'
    encontrado=0
    iteraciones=0
    for numeros in permutations(lista_numeros,5):
        if(encontrado):
            break
        for signos in permutations(lista_signos,4):
            expresion=str(numeros[0])+signos[0]+str(numeros[1])+signos[1]+str(numeros[2])+signos[2]+str(numeros[3])+signos[3]+str(numeros[4])
            valor=eval(expresion)
            iteraciones+=1
            if(valor==numero_deseado):
                encontrado=1
                expresion_definitiva=expresion
                break
    if(encontrado):
        return iteraciones,expresion_definitiva
    else:
        return iteraciones,'No se puede representar el numero deseado con las restricciones dadas'


In [ ]:
###Ejemplo de aplicación

lista_numeros=[1,2,3,4,5,6,7,8,9]
print(algoritmo_fuerza_bruta(lista_numeros,77))
print(algoritmo_fuerza_bruta(lista_numeros,36))
print(algoritmo_fuerza_bruta(lista_numeros,0))

(242511, '7/1-2+8*9')
(42302, '2/1+5*8-6')
(268, '1+2/3*6-5')


Calcula la complejidad del algoritmo por fuerza bruta

El algoritmo tiene una complejidad factorial, debido a su estructura ramificada y las permutaciones posibles.

# 5.  (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

En este primer ejemplo se implementa el algoritmo de fuerza bruta pero desordenando la lista de números de entrada. Esto proporciona puntos de partida aleatorios, ya que la función que lleva a cabo las permutaciones sigue el orden de la lista. Desordenando esta aleatoriamente, en cada ejecución se tiene un punto de partida y en ocasiones puede ser cercano al deseado. En la implementación anterior, el número de iteraciones necesarias para cada combinación era fija ya que se seguía el orden establecido. Como ventaja ahora puede que ese número sea inferior, aunque por otra parte también puede empeorar.

### 5.1  Algoritmo de fuerza bruta modificado


In [ ]:

def algoritmo_fuerza_bruta_shuffle(lista_numeros,numero_deseado):
    
    random.shuffle(lista_numeros)
    lista_signos='+-/*'
    encontrado=0
    iteraciones=0
    for numeros in permutations(lista_numeros,5):
        if(encontrado):
            break
        for signos in permutations(lista_signos,4):
            expresion=str(numeros[0])+signos[0]+str(numeros[1])+signos[1]+str(numeros[2])+signos[2]+str(numeros[3])+signos[3]+str(numeros[4])
            valor=eval(expresion)
            iteraciones+=1
            if(valor==numero_deseado):
                encontrado=1
                expresion_definitiva=expresion
                break
    if(encontrado):
        return iteraciones,expresion_definitiva
    else:
        return iteraciones,'No se puede representar el numero deseado con las restricciones dadas'

    

In [ ]:
lista_numeros=[1,2,3,4,5,6,7,8,9]
numeros_introducidos=[77,36,0,-4,-8,-54]
pruebas=[]
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_fuerza_bruta_shuffle(lista_numeros,numeros_introducidos[i]))


for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))

Numero deseado: 77 Iteraciones: 102670 Expresión: 9*8-4/2+7
Numero deseado: 36 Iteraciones: 330 Expresión: 6/1*7-8+2
Numero deseado: 0 Iteraciones: 492 Expresión: 3-2*8/4+1
Numero deseado: -4 Iteraciones: 74 Expresión: 4+7-5*6/2
Numero deseado: -8 Iteraciones: 324 Expresión: 4-9*2/1+6
Numero deseado: -54 Iteraciones: 101848 Expresión: 8/2-7*9+5


### 5.2 Algoritmo de ramificación y poda

El algoritmo diseñado tiene dos procedimientos según si el número es positivo o igual que 0 o si es negativo. En primer lugar, se explica el procedimiento de los números positivos:

- El **primer nodo** a explorar se obtiene generando un número aleatorio y diviendo el número deseado por el usuario entre dicho valor aleatorio. Si la parte entera de esta división es un número de la lista y es distinto que el primer elemento generado aleatoriamente, se continúa con el algoritmo. En caso contrario, se siguen explorando opciones eliminando la ya evaluada, hasta que se encuentra una viable. 
- **Ejemplo positivo**: Número deseado: 50, primer elemento generado aleatoriamente 8, división entera 6, se continúa con el algoritmo con la cadena 6*8
- **Ejemplo negativo 1**: Número deseado: 50, primer elemento generado aleatoriamente 1, división entera 50, no se puede continuar porque 50 no está entre los números posibles.
- **Ejemplo negativo 2**: Número deseado: 50, primer elemento generado aleatoriamente 7, división entera 7. Puesto que el número es repetido, en estos casos se suma un valor de uno al segundo número para que sea distinto y se comprueba si es válido para continuar.
- En la **siguiente etapa** , se eliminan los números seleccionados anteriormente y se continúa con los disponibles. El siguiente signo de la cadena puede ser un "+" o un "-", eligiéndose aleatoriamente uno de ellos y continuando con dicha elección. 
- Una vez seleccionado el signo, se continúa con las **opciones restantes**. En esta etapa se asignan los valores que van acompañados del operador **división**. Es por ello que si la división no es entera, no se tiene en cuenta directamente.
- Una vez seleccionado los elementos que componen la división, se comprueba que el valor restante sea posible de alcanzar con los elementos restantes. Si es así, se devuelve la **cadena final**.

In [ ]:
###Algoritmo ramificación y poda
import random
from itertools import permutations

def algoritmo_ramificacion_poda(lista_numeros,numero_deseado):
    iteraciones = 0
    cadena_encontrada = 0
    lista_original=lista_numeros
    while (cadena_encontrada == 0): #Ejecuta el bucle mientras no encuentre la expresión
        lista_numeros = lista_original.copy()
        
        if (numero_deseado >= 0): #Si el número es positivo o 0
            iteraciones += 1
            lista_numeros_aux = lista_numeros.copy() #Lista auxiliar de la que se irán eliminando opciones ya exploradas
            primer_numero = random.randint(1, len(lista_numeros)) #Primer número aleatorio
            segundo_numero = int(numero_deseado / primer_numero)
            
            #En este bucle se comprueba que el primer y segundo elemento son válidos, como se ha ilustrado con los ejemplos
            while ((segundo_numero not in lista_numeros) or primer_numero==segundo_numero):
                if (segundo_numero == primer_numero or segundo_numero==0):
                    segundo_numero += 1
                    iteraciones += 1
                else:
                    lista_numeros_aux.remove(primer_numero)
                    primer_numero = lista_numeros_aux[random.randint(0, len(lista_numeros_aux) - 1)]
                    segundo_numero = int(numero_deseado / primer_numero)
                    iteraciones += 1

            cadena = str(primer_numero) + '*' + str(segundo_numero)
            lista_numeros.remove(primer_numero) #Eliminación del primer elemento de la lista de números disponibles
            lista_numeros.remove(segundo_numero)#Eliminación del segundo elemento de la lista de números disponibles
            numeros_restantes = lista_numeros.copy()
            
            #Se genera el número aleatorio que sirve para seleccionar el "+" o el "-"
            aleatorio_signo = random.randint(1, 2) 
            if (aleatorio_signo == 1):
                signo = '+'
                ultimo_signo = '-'
            else:
                signo = '-'
                ultimo_signo = '+'
                
            #Se continúa construyendo la cadena con las opciones restantes
            for numeros in permutations(numeros_restantes, 2):
                primer_elemento = numeros[0]
                segundo_elemento = numeros[1]
                iteraciones+=1
                
                if (primer_elemento % segundo_elemento == 0): #Si la división no es entera, no se tiene encuenta
                    cadena_aux = cadena + signo + str(primer_elemento) + '/' + str(segundo_elemento)
                    valor_restante = numero_deseado - eval(cadena_aux)
                    
                    #Si el último signo hace imposible alcanzar el valor restante, no se tiene en cuenta la combinación
                    if ((valor_restante >= 0 and ultimo_signo == '-') or (valor_restante <= 0 and ultimo_signo == '+')):
                        pass
                    else: #En caso de que sí que sea válido
                        numeros_restantes_2 = numeros_restantes.copy()
                        numeros_restantes_2.remove(primer_elemento) #Se quitan los elementos asignados a la división
                        numeros_restantes_2.remove(segundo_elemento)#Se quitan los elementos asignados a la división
                        if (abs(valor_restante) in numeros_restantes_2): #Si el valor restante se encuentra en la lista de elementos disponibles, se selecciona este
                            ultimo_valor = valor_restante
                            cadena_final = cadena_aux + ultimo_signo + str(abs(int(valor_restante)))
                            return iteraciones,cadena_final
        #El funcionamiento de la parte negativa es análogo al de la posisitiva.
        #La única diferencia es que no se escoge signo "+" o "-" ya que el "-" siempre se ubica al inicio
        else:
            iteraciones += 1
            lista_numeros_aux = lista_numeros.copy()
            primer_numero = random.randint(1, len(lista_numeros_aux))
            segundo_numero = int(abs(numero_deseado) / primer_numero)
            while ((segundo_numero not in lista_numeros) or primer_numero == segundo_numero or abs(segundo_numero*primer_numero)<=abs(numero_deseado)):
                if (segundo_numero == primer_numero):
                    segundo_numero += 1
                    iteraciones += 1
                else:
                    primer_numero = random.randint(1, len(lista_numeros_aux))
                    segundo_numero = int(abs(numero_deseado) / primer_numero)+1
                    iteraciones += 1

            cadena = '-'+str(primer_numero) + '*' + str(segundo_numero)
            lista_numeros.remove(primer_numero)
            lista_numeros.remove(segundo_numero)
            numeros_restantes = lista_numeros.copy()
            ultimo_signo = '+'
            for numeros in permutations(numeros_restantes, 2):
                primer_elemento = numeros[0]
                segundo_elemento = numeros[1]
                iteraciones += 1
                if (primer_elemento % segundo_elemento == 0):
                    cadena_aux = cadena + ultimo_signo + str(primer_elemento) + '/' + str(segundo_elemento)
                    if(eval(cadena_aux)==numero_deseado):
                        return iteraciones,cadena_aux







Se lleva a cabo la misma prueba que con los algoritmos anteriores para comprobar cuantas iteraciones necesita

In [ ]:
lista_numeros=[1,2,3,4,5,6,7,8,9]
numeros_introducidos=[77,36,0,-4,-8,-54]
pruebas=[]
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_ramificacion_poda(lista_numeros,numeros_introducidos[i]))


for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
                

Numero deseado: 77 Iteraciones: 44 Expresión: 9*8+7/1-2
Numero deseado: 36 Iteraciones: 27 Expresión: 4*9-6/2+3
Numero deseado: 0 Iteraciones: 110 Expresión: 1*2+6/3-4
Numero deseado: -4 Iteraciones: 78 Expresión: -3*2+8/4
Numero deseado: -8 Iteraciones: 35 Expresión: -3*4+8/2
Numero deseado: -54 Iteraciones: 9 Expresión: -8*7+2/1


### 5.3 Comparativa de tiempos de ejecución de todos los algoritmos

In [ ]:
from time import time
import numpy as np

lista_numeros=[1,2,3,4,5,6,7,8,9]
numeros_introducidos=[77,36,0,-4,-8,-54]
pruebas=[]
print('Algoritmo fuerza bruta:\n')

start=time()
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_fuerza_bruta(lista_numeros,numeros_introducidos[i]))
final=time()

for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
print('Tiempo empleado: '+str(np.round(final-start,2)))

Algoritmo fuerza bruta:

Numero deseado: 77 Iteraciones: 242511 Expresión: 7/1-2+8*9
Numero deseado: 36 Iteraciones: 42302 Expresión: 2/1+5*8-6
Numero deseado: 0 Iteraciones: 268 Expresión: 1+2/3*6-5
Numero deseado: -4 Iteraciones: 340 Expresión: 1+2/3*6-9
Numero deseado: -8 Iteraciones: 2771 Expresión: 1-2*6+9/3
Numero deseado: -54 Iteraciones: 83269 Expresión: 3/1+6-7*9
Tiempo empleado: 2.8


In [ ]:
from time import time
import numpy as np

lista_numeros=[1,2,3,4,5,6,7,8,9]
numeros_introducidos=[77,36,0,-4,-8,-54]
pruebas=[]
print('Algoritmo fuerza bruta shuffle:\n')

start=time()
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_fuerza_bruta_shuffle(lista_numeros,numeros_introducidos[i]))
final=time()

for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
print('Tiempo empleado: '+str(np.round(final-start,2)))

Algoritmo fuerza bruta shuffle:

Numero deseado: 77 Iteraciones: 56395 Expresión: 8*9+7-2/1
Numero deseado: 36 Iteraciones: 902 Expresión: 2/1+5*8-6
Numero deseado: 0 Iteraciones: 4 Expresión: 4+6/3*2-8
Numero deseado: -4 Iteraciones: 5940 Expresión: 1-4*6/2+7
Numero deseado: -8 Iteraciones: 98 Expresión: 1+5-8*7/4
Numero deseado: -54 Iteraciones: 23629 Expresión: 8/2+5-7*9
Tiempo empleado: 0.7


In [ ]:
from time import time
import numpy as np

lista_numeros=[1,2,3,4,5,6,7,8,9]
numeros_introducidos=[77,36,0,-4,-8,-54]
pruebas=[]
print('Algoritmo ramificación y poda:\n')

start=time()
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_ramificacion_poda(lista_numeros,numeros_introducidos[i]))
final=time()

for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
print('Tiempo empleado: '+str(final-start))

Algoritmo ramificación y poda:

Numero deseado: 77 Iteraciones: 12 Expresión: 9*8-2/1+7
Numero deseado: 36 Iteraciones: 27 Expresión: 9*4+6/2-3
Numero deseado: 0 Iteraciones: 65 Expresión: 2*1+6/3-4
Numero deseado: -4 Iteraciones: 34 Expresión: -2*3+8/4
Numero deseado: -8 Iteraciones: 22 Expresión: -5*2+6/3
Numero deseado: -54 Iteraciones: 17 Expresión: -7*8+2/1
Tiempo empleado: 0.0009644031524658203


### 5.4 Comparativa con 20 números posibles a elegir
Se recuerda que en base al cálculo realizado en el primer apartado, con 20 números se tienen **44651520 posibilidades** por fuerza bruta.

In [ ]:
from time import time
import numpy as np

lista_numeros=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
numeros_introducidos=[210,100,0,-4,-8,-54]
pruebas=[]
print('Algoritmo fuerza bruta:\n')

start=time()
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_fuerza_bruta(lista_numeros,numeros_introducidos[i]))
final=time()

for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
print('Tiempo empleado: '+str(np.round(final-start,2)))

Algoritmo fuerza bruta:

Numero deseado: 210 Iteraciones: 1277189 Expresión: 1+12*18-14/2
Numero deseado: 100 Iteraciones: 176551 Expresión: 1-3+12/2*17
Numero deseado: 0 Iteraciones: 176 Expresión: 1-2+3*4/12
Numero deseado: -4 Iteraciones: 868 Expresión: 1+2/3*6-9
Numero deseado: -8 Iteraciones: 894 Expresión: 1+2*3/6-10
Numero deseado: -54 Iteraciones: 39505 Expresión: 1+2-9/3*19
Tiempo empleado: 11.11


In [ ]:
from time import time
import numpy as np

lista_numeros=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
numeros_introducidos=[210,100,0,-4,-8,-54]
pruebas=[]
print('Algoritmo fuerza bruta shuffle:\n')

start=time()
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_fuerza_bruta_shuffle(lista_numeros,numeros_introducidos[i]))
final=time()

for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
print('Tiempo empleado: '+str(np.round(final-start,2)))

Algoritmo fuerza bruta shuffle:

Numero deseado: 210 Iteraciones: 140454 Expresión: 4+13*17/1-15
Numero deseado: 100 Iteraciones: 28399 Expresión: 20-8+16/2*11
Numero deseado: 0 Iteraciones: 2530 Expresión: 2-7/5*15+19
Numero deseado: -4 Iteraciones: 354 Expresión: 1/2*14-19+8
Numero deseado: -8 Iteraciones: 5148 Expresión: 10-14*5/2+17
Numero deseado: -54 Iteraciones: 20449 Expresión: 7+17-18/3*13
Tiempo empleado: 1.52


In [ ]:
from time import time
import numpy as np

lista_numeros=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
numeros_introducidos=[210,100,0,-4,-8,-54]
pruebas=[]
print('Algoritmo ramificacion y poda:\n')

start=time()
for i in range(len(numeros_introducidos)):
    pruebas.append(algoritmo_ramificacion_poda(lista_numeros,numeros_introducidos[i]))
final=time()

for i in range(len(numeros_introducidos)):
    print('Numero deseado: '+str(numeros_introducidos[i])+' Iteraciones: '+str(pruebas[i][0])+' Expresión: '+str(pruebas[i][1]))
print('Tiempo empleado: '+str(final-start))

Algoritmo ramificacion y poda:

Numero deseado: 210 Iteraciones: 90 Expresión: 15*14-6/2+3
Numero deseado: 100 Iteraciones: 257 Expresión: 17*5+18/1-3
Numero deseado: 0 Iteraciones: 566 Expresión: 6*1-18/2+3
Numero deseado: -4 Iteraciones: 71 Expresión: -7*1+6/2
Numero deseado: -8 Iteraciones: 397 Expresión: -4*3+8/2
Numero deseado: -54 Iteraciones: 71 Expresión: -20*3+6/1
Tiempo empleado: 0.0009949207305908203


Queda probada la eficacia del algoritmo, ya que ha conseguido gracias a la selección de nodos de interés ser eficiente sin apreciar un notable incremento de coste computacional incluso elevando al doble la lista de posibles números a escoger.

# 6. (*)Calcula la complejidad del algoritmo 

El nuevo algoritmo tiene complejidad lineal, aunque tenga permutaciones reduce mucho la búsqueda con la selección de nodosd e interés.

# 7. Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

En este problema no tiene tanto sentido generar un juego de datos de entrada aleatorio. Lo que se ha hecho para trabajar este apartado es diseñar el algoritmo de fuerza bruta shuffle. 

 # 8. Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Creo que sería interesante estudiar la posibilidad de descartar algunas permutaciones que se sabe que no van a ser de interés. Por ejemplo, si se prueba la combinación para la división 1/2 se sabe que no es válida ni tampoco lo serán 1/3, 1/4 .. ya que está garantizado que no será una división entera. Se podría estudiar la posibilidad de que el bucle for no planteara la posibilidad de esas permutaciones, aunque en el algoritmo diseñado se comprueba que no son válidas y no se tienen en cuenta.